Iris Classification with Neural Networks in Haskell
===================================================

This program demonstrates how to build a multiclass classification model
in Haskell using the DataFrame and Hasktorch libraries.

What This Program Does
----------------------

1. Loads the famous Iris dataset (flower measurements and species)
2. Splits the data into training (70%) and test (30%) sets
3. Builds a 3-layer neural network (4 inputs → 8 hidden → 3 outputs)
4. Trains the model for 10,000 epochs using gradient descent
5. Evaluates performance with confusion matrices and classification metrics



## Loading and analysing the data

We'll begin by loading and sampling the data to get a rough sense of what's in our dataset.

In [ ]:
import qualified DataFrame as D

df <- D.readParquet "./data/iris.parquet"

D.take 5 df

--------------------------------------------------------------------------------------------------------------------------------
index<br>Int | sepal.length<br>Double | sepal.width<br>Double | petal.length<br>Double | petal.width<br>Double | variety<br>Text
-------------|------------------------|-----------------------|------------------------|-----------------------|----------------
0            | 5.1                    | 3.5                   | 1.4                    | 0.2                   | Setosa         
1            | 4.9                    | 3.0                   | 1.4                    | 0.2                   | Setosa         
2            | 4.7                    | 3.2                   | 1.3                    | 0.2                   | Setosa         
3            | 4.6                    | 3.1                   | 1.5                    | 0.2                   | Setosa         
4            | 5.0                    | 3.6                   | 1.4                    | 0.2                   | Setosa         


We have 4 features and a target variable. The features are measurements of parts of flowering plants. The petal (as you probbaly know) is the colourful part of the plant and the sepal is the green, protective casing around the petal.

We already know from the types of the columns that we have no missing values. If we did have any missing values we'd have a column that starts with `Maybe`. Not having missing values saves us having to do some data cleaning meaning we can jump straight into trying to understand the data.


### How many of each kind of plant are in the dataset?

If the dataset is very imbalanced it might be difficult to accurately predict the variety from the features (one variety could have only one example). In light of that, we must first check that our dataset is balanced.

In [2]:
:set -XOverloadedStrings

import qualified DataFrame.Display.Web.Plot as Plt

Plt.plotPie "variety" Nothing df

<canvas id="chart_NfGqVBJVDF583EqVlYoHm1lHtBtDTBkgtQ35JJvjTTZEbURM" style="width:100%;max-width:600px;height:400px"></canvas>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.9.4/Chart.js"></script>
<script>
new Chart("chart_NfGqVBJVDF583EqVlYoHm1lHtBtDTBkgtQ35JJvjTTZEbURM", {
  type: "pie",
  data: {
    labels: ["Setosa","Versicolor","Virginica"],
    datasets: [{
      data: [50.0,50.0,50.0],
      backgroundColor: ["rgb(255, 99, 132)","rgb(54, 162, 235)","rgb(255, 206, 86)"]
    }]
  },
  options: {
    title: { display: true, text: "variety" }
  }
});
</script>

Great! A perfect balance. Now we'd like to investigate if we can engineer our features to be more discriminative before we pass them to a machine learning model. We can start by looking at how each of thes features is distributed for the different varieties.

In [3]:
features = D.exclude ["variety"] df

Plt.plotStackedBars "variety" (D.columnNames features) df

<canvas id="chart_XStXPLy8wzGZzixdxA5Mua8BE7tXC9FYDUAGBLx6SMx8QXLzvr" style="width:100%;max-width:600px;height:400px"></canvas>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.9.4/Chart.js"></script>
<script>
new Chart("chart_XStXPLy8wzGZzixdxA5Mua8BE7tXC9FYDUAGBLx6SMx8QXLzvr", {
  type: "bar",
  data: {
    labels: ["Setosa","Versicolor","Virginica"],
    datasets: [
    {
      label: "sepal.length",
      data: [250.29999999999998,296.8,329.3999999999999],
      backgroundColor: "rgb(255, 99, 132)"
    },
    {
      label: "sepal.width",
      data: [171.40000000000003,138.50000000000003,148.7],
      backgroundColor: "rgb(54, 162, 235)"
    },
    {
      label: "petal.length",
      data: [73.10000000000001,212.99999999999997,277.59999999999997],
      backgroundColor: "rgb(255, 206, 86)"
    },
    {
      label: "petal.width",
      data: [12.299999999999995,66.3,101.29999999999998],
      backgroundColor: "rgb(75, 192, 192)"
    }
    ]
  },
  options: {
    title: { display: true, text: "Stacked Bar Chart" },
    scales: {
      xAxes: [{ stacked: true }],
      yAxes: [{ stacked: true, ticks: { beginAtZero: true } }]
    }
  }
});
</script>

This chart shows us all the sums of the features for each variety. Since the classes are balanced this chart will track the shape of the averages.

The chart is interactive so we can select and deselect each feature as we see fit.

From a quick glance we can already tell that Virginica tends to be the largest while Setosa is the smallest.

Looking more closely at each of the features we see that the order of magnitudes tends to be `Virginica > Versicolor > Setosa`. In a surprising turn, however, for sepal width Setosa is the largest.


Averages conceal a lot of information so we should also look at the distributions of each of these features for the varieties. We can do this with a box plot.

In [4]:
import Data.Text (Text)
boxForVariety var = D.plotBoxPlots (D.columnNames (D.selectBy [D.byProperty D.isNumeric] df)) (D.filter "variety" (== var) df)

mapM_ boxForVariety ["Setosa" :: Text, "Versicolor", "Virginica"]

   6.4│                                                            
      │                                                            
      │                                     ┴                      
      │                                     │                      
      │                                  ═══════                   
      │                                  │─────│                   
      │                                     ┬                ┴     
      │                                                      │     
      │                                                   │─────│  
      │                                                   ═══════  
   3.2│                                                   │─────│  
      │                                                      │     
      │                                                      ┬     
      │                                                            
      ││──┴──│                                  

Setosas are likely the easiest to identify. Their petal length and widths tend to be very small with little variance. Versicolors and Virginicas might be a little more difficult to tell apart since they have similar dimensions.

But we can already tell from the box plot that the ratio of the areas of the sepal to the petal in Virginicas is much smaller than the same ratio is Versicolors.

From our exploration it seems out dataset have 5 feature candidates:

* sepal area
* sepal length to width ratio
* petal area
* petal length to width ratio
* sepal area / petal area

To derive these feature it'll help to create typed references to the columns. 

In [5]:
:set -XTemplateHaskell
import qualified DataFrame.Functions as F

F.declareColumns df

Now we have the features defined as expressions we can use in our computations.

In [6]:
import DataFrame ((|>))

engineered = df
               |> D.derive "sepal_area" (sepal_length * sepal_width)
               |> D.derive "sepal_ratio" (sepal_length / sepal_width)
               |> D.derive "petal_area" (petal_length * petal_width)
               |> D.derive "petal_ratio" (petal_length / petal_width)
               |> D.derive "area_ratio" ((sepal_length * sepal_width) / (petal_length * petal_width))

Now let's repeat the previous analysis with out new features.

In [7]:
Plt.plotStackedBars "variety" (D.columnNames (D.exclude ["variety"] engineered)) engineered

<canvas id="chart_ZYHajOx7a7wG4g57jbBE9BO6veLPc6HPb1wdZDTJeLDIwRPMiG6S" style="width:100%;max-width:600px;height:400px"></canvas>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.9.4/Chart.js"></script>
<script>
new Chart("chart_ZYHajOx7a7wG4g57jbBE9BO6veLPc6HPb1wdZDTJeLDIwRPMiG6S", {
  type: "bar",
  data: {
    labels: ["Setosa","Versicolor","Virginica"],
    datasets: [
    {
      label: "sepal.length",
      data: [250.29999999999998,296.8,329.3999999999999],
      backgroundColor: "rgb(255, 99, 132)"
    },
    {
      label: "sepal.width",
      data: [171.40000000000003,138.50000000000003,148.7],
      backgroundColor: "rgb(54, 162, 235)"
    },
    {
      label: "petal.length",
      data: [73.10000000000001,212.99999999999997,277.59999999999997],
      backgroundColor: "rgb(255, 206, 86)"
    },
    {
      label: "petal.width",
      data: [12.299999999999995,66.3,101.29999999999998],
      backgroundColor: "rgb(75, 192, 192)"
    },
    {
      label: "sepal_area",
      data: [862.8900000000003,826.3100000000001,984.2299999999999],
      backgroundColor: "rgb(153, 102, 255)"
    },
    {
      label: "sepal_ratio",
      data: [73.50938405113742,108.02010958439155,111.52263694471121],
      backgroundColor: "rgb(255, 159, 64)"
    },
    {
      label: "petal_area",
      data: [18.280000000000012,286.02,564.8099999999997],
      backgroundColor: "rgb(255, 99, 132)"
    },
    {
      label: "petal_ratio",
      data: [345.4,162.1418466337584,139.03311692002234],
      backgroundColor: "rgb(54, 162, 235)"
    },
    {
      label: "area_ratio",
      data: [2846.2560065966027,148.52885354259752,88.25571762288054],
      backgroundColor: "rgb(255, 206, 86)"
    }
    ]
  },
  options: {
    title: { display: true, text: "Stacked Bar Chart" },
    scales: {
      xAxes: [{ stacked: true }],
      yAxes: [{ stacked: true, ticks: { beginAtZero: true } }]
    }
  }
});
</script>

Petal ratio and sepal ratio don't help us distinguish Versicolor from Virginica. Sepal area, petal area, and area ratio seem to give us a pretty good edge. 

Those seem like good candidates for the features to include in the model. Let's look at the box plots to make sure.

In [8]:
boxForVariety var = D.plotBoxPlots ["petal_area", "sepal_area", "area_ratio"] (D.filter "variety" (== var) engineered)

mapM_ boxForVariety ["Setosa" :: Text, "Versicolor", "Virginica"]

 156.3│                                                            
      │                                                            
      │                                                       ┴    
      │                                                       │    
      │                                                       │    
      │                                                       │    
      │                                                       │    
      │                                                       │    
      │                                                       │    
      │                                                       │    
  78.2│                                                       │    
      │                                                  │────────│
      │                                                  │        │
      │                                                  ══════════
      │                                         

The features seem great for modelling. Let's create our final dataframe with just these features.

In [9]:
modellingDf = D.select ["petal_area", "sepal_area", "area_ratio", "variety"] engineered

Defining Our Data Types
------------------------

In Haskell, we can use the type system to represent our data precisely.
The Iris dataset contains three species of flowers, which we represent
as an algebraic data type (similar to an enum in other languages):



In [10]:
import qualified Data.Text as T

data Iris
    = Setosa
    | Versicolor
    | Virginica
    deriving (Eq, Show, Read, Ord, Enum)

withTypedLabel =
        modellingDf
            |> D.derive
                "variety"
                (F.lift (fromEnum . read @Iris . T.unpack) (F.col "variety"))

The `deriving` clause automatically generates useful functions:
- `Eq`: Allows us to compare Iris values for equality
- `Show`: Converts Iris to a String (e.g., "Setosa")
- `Read`: Converts a String to Iris (e.g., "Setosa" → Setosa)
- `Ord`: Allows ordering/sorting
- `Enum`: Lets us convert to/from integers (Setosa=0, Versicolor=1, Virginica=2)


### Creating the training and test data

To ensure that our model generalizes well we will split out data into train and test sets. We can do this with the `randomSplit` function.

After we split the data we can convert it to Torch tensors so we can train on it.

In [11]:
import qualified DataFrame.Hasktorch as DHT
import qualified System.Random as SysRand
import Control.Exception (throw)

let (trainDf, testDf) = D.randomSplit (SysRand.mkStdGen 42) 0.7 withTypedLabel

let trainFeaturesTr =
        trainDf
            |> D.exclude ["variety"]
            |> DHT.toTensor
let testFeaturesTr =
        testDf
            |> D.exclude ["variety"]
            |> DHT.toTensor

let trainLabels = either throw id (D.columnAsIntVector "variety" trainDf)
let testLabels = either throw id (D.columnAsIntVector "variety" testDf)

Since we are predicting one of many classes, out target should be a one-hot vector:

- 0 (Setosa) → [1.0, 0.0, 0.0]
- 1 (Versicolor) → [0.0, 1.0, 0.0]
- 2 (Virginica) → [0.0, 0.0, 1.0]

In [12]:
import qualified Torch as HT

let trainLabelsTr = HT.toType HT.Float $ HT.oneHot 3 $ HT.asTensor $ trainLabels
let testLabelsTr = HT.toType HT.Float $ HT.oneHot 3 $ HT.asTensor $ testLabels

We're now read to define our neural network!


Neural Network Architecture
----------------------------

We define our Multi-Layer Perceptron (MLP) architecture in two parts:

First, a specification that describes the shape of our network:

In [13]:
data MLPSpec = MLPSpec
    { inputFeatures :: Int   -- Number of input features (3 for our dataset)
    , hiddenFeatures :: Int  -- Number of neurons in hidden layer
    , outputFeatures :: Int  -- Number of output classes (3 species)
    }
    deriving (Show, Eq)

Second, the actual model with its layers. Each layer is a `Linear`
transformation (like `nn.Linear` in PyTorch):

In [14]:
:set -XDeriveGeneric
import qualified Torch as HT

import GHC.Generics (Generic)

data MLP = MLP
    { l0 :: HT.Linear  -- Input → Hidden layer
    , l1 :: HT.Linear  -- Hidden → Output layer
    }
    deriving (Generic, Show)

Network Architecture Diagram:

    Input Layer (4)  →  Hidden Layer (8)  →  Output Layer (3)
    ---------------     -----------------     ----------------
    sepal_area          ReLU activation       Softmax
    petal_area          (introduces           (produces
    area_ratio          non-linearity)        probabilities)
                                              Setosa
                                              Versicolor
                                              Virginica

Making Our Model Trainable
---------------------------

We need to tell Hasktorch how to initialize our network with random weights.
This is similar to defining `__init__()` in a PyTorch `nn.Module`:



In [15]:
:set -XRecordWildCards

instance HT.Parameterized MLP
instance HT.Randomizable MLPSpec MLP where
    sample MLPSpec{..} =
        MLP
            <$> HT.sample (HT.LinearSpec inputFeatures hiddenFeatures)
            <*> HT.sample (HT.LinearSpec hiddenFeatures outputFeatures)


The `<$>` and `<*>` operators are Haskell's way of working with random
initialization. Think of this as: "Create an MLP by randomly sampling
weights for both layers."


Forward Pass
------------

This function defines how data flows through the network. It's equivalent
to the `forward()` method in PyTorch. Read it from right to left (or
bottom to top in the chain):



In [16]:
mlp :: MLP -> HT.Tensor -> HT.Tensor
mlp MLP{..} =
    HT.softmax (HT.Dim 1)       -- 4. Apply softmax (probabilities sum to 1)
        . HT.linear l1          -- 3. Apply second linear layer
        . HT.relu               -- 2. Apply ReLU activation
        . HT.linear l0          -- 1. Apply first linear layer

In Python/PyTorch, this would look like:
```python
def forward(self, x):
    x = self.l0(x)
    x = F.relu(x)
    x = self.l1(x)
    x = F.softmax(x, dim=1)
    return x
```


Training Loop
-------------

This is our main training function. It's similar to the epoch loop in
PyTorch training code:



In [17]:
import Control.Monad (when)

trainLoop ::
    Int ->                          -- Number of epochs
    (HT.Tensor, HT.Tensor) ->       -- Training features and labels
    (HT.Tensor, HT.Tensor) ->       -- Test features and labels
    MLP ->                          -- Initial model
    IO MLP                          -- Returns trained model
trainLoop
    n
    (features, labels)
    (testFeatures, testLabels)
    initialM = do
        let patience = 5  -- stop after 5 checks without improvement
            checkInterval = 500
            initialBestLoss = read @Float "Infinity"
            
        -- Extended state: (modelState, bestModelState, bestLoss, patienceCounter)
        let extendedInitialState = (initialM, initialM, initialBestLoss, 0)
        
        (_, bestModel, _, _) <- HT.foldLoop extendedInitialState n $ \(state, bestState, bestLoss, counter) i -> do
            -- Early stopping check
            if counter >= patience
                then pure (state, bestState, bestLoss, counter)
                else do
                    -- Forward pass: compute predictions
                    let predicted = mlp state features
                    
                    -- Compute loss (how wrong our predictions are)
                    let loss = HT.binaryCrossEntropyLoss' labels predicted
                    
                    -- Backward pass: update weights using gradient descent
                    (state', _) <- HT.runStep state HT.GD loss 1e-2
                    
                    let testPredicted = mlp state' testFeatures
                        testLoss = HT.binaryCrossEntropyLoss' testLabels testPredicted
                        currentTestLoss = HT.asValue testLoss :: Float

                    when (i `mod` checkInterval == 0) $ do
                        putStrLn $
                            "Iteration: "
                                ++ show i
                                ++ " | Training Set Loss: "
                                ++ show (HT.asValue loss :: Float)
                                ++ " | Test Set Loss: "
                                ++ show currentTestLoss
                    
                    -- Update best model and patience counter
                    let (newBestState, newBestLoss, newCounter) =
                            if currentTestLoss < bestLoss
                                then (state', currentTestLoss, 0)
                                else (bestState, bestLoss, counter + 1)
                    
                    pure (state', newBestState, newBestLoss, newCounter)

        pure bestModel

Initialize and train the neural network
=======================================

Create a random initial model with:
- 3 input neurons (one for each feature)
- 8 hidden neurons (arbitrary choice, can be tuned)
- 3 output neurons (one for each species)

In [18]:
initialModel <- HT.sample $ MLPSpec 3 8 3

trainedModel <-
    trainLoop
        10000
        (trainFeaturesTr, trainLabelsTr)
        (testFeaturesTr, testLabelsTr)
        initialModel

putStrLn "Your model weights are given as follows: "
print trainedModel

Iteration: 500 | Training Set Loss: 7.246852e-2 | Test Set Loss: 0.115057826

Your model weights are given as follows:

MLP {l0 = Linear {weight = IndependentTensor {toDependent = Tensor Float [8,3] [[-0.4836   , -1.8817e-2, -0.4929   ],
                    [-0.5145   ,  0.7143   , -0.4100   ],
                    [ 0.8047   , -0.1833   , -0.1164   ],
                    [ 0.3303   , -0.2834   , -0.2514   ],
                    [ 0.5591   ,  0.1656   ,  0.4565   ],
                    [ 0.4366   , -0.2755   ,  0.5811   ],
                    [-0.1693   ,  3.6658e-2,  0.6852   ],
                    [-0.5325   , -0.2528   ,  0.2066   ]]}, bias = IndependentTensor {toDependent = Tensor Float [8] [-0.3102   ,  0.2562   ,  7.1749e-2,  0.2139   , -0.5473   ,  0.1614   ,  0.1399   ,  0.4536   ]}}, l1 = Linear {weight = IndependentTensor {toDependent = Tensor Float [3,8] [[-0.1612   , -0.5567   , -6.2411e-2, -0.2086   , -0.2038   ,  0.3749   ,  0.1930   , -2.5730e-2],
                    [-7.5593e-2,  0.3473   , -0.4055   , -0.3468   , -0.4487   , -0.2369   ,  0.4174   , -4.2022e-2],
                    [-0.20

Evaluation Metrics
------------------

We define a confusion matrix type to track our predictions:



In [19]:
import qualified Data.Array as A

type ConfusionMatrix = A.Array (Int, Int) Float


The confusion matrix shows actual vs predicted labels:

                Predicted
              0     1     2
    Actual 0  TP    FN    FN
           1  FP    TP    FN
           2  FP    FP    TP

where rows are actual labels and columns are predicted labels.



In [20]:
confusionMatrix :: Int -> [Int] -> [Int] -> ConfusionMatrix
confusionMatrix n actuals preds = A.accumArray (+) 0 bnds [(x, 1) | x <- zip actuals preds]
  where
    bnds = ((0, 0), (n - 1, n - 1))


Helper function to print the confusion matrix in a readable format:



In [21]:
import Control.Monad (unless)
import Text.Printf (printf)

pprintMatrix :: ConfusionMatrix -> String
pprintMatrix mtx =
    unlines $
        header
            : [ unwords $ printf "%5d" y : [printf "%5.2f" (mtx A.! (x, y)) | x <- [x1 .. x2]]
              | y <- [y1 .. y2]
              ]
  where
    ((x1, y1), (x2, y2)) = A.bounds mtx
    header = unwords $ "     " : [printf "%5d" x | x <- [x1 .. x2]]


Calculate sums for precision and recall:



In [22]:
rowSumAt :: ConfusionMatrix -> Int -> Float
rowSumAt mtx n = sum [x | ((i, j), x) <- A.assocs mtx, j == n]

colSumAt :: ConfusionMatrix -> Int -> Float
colSumAt mtx n = sum [x | ((i, j), x) <- A.assocs mtx, i == n]


Precision = True Positives / (True Positives + False Positives)
(Of all the times we predicted class X, how often were we right?)



In [23]:
classwisePrecision :: ConfusionMatrix -> [Float]
classwisePrecision mtx = [mtx A.! (i, i) / mtx `rowSumAt` i | i <- [y1 .. y2]]
  where
    ((x1, y1), (x2, y2)) = A.bounds mtx


Recall = True Positives / (True Positives + False Negatives)
(Of all the actual class X examples, how many did we find?)



In [24]:
classwiseRecall :: ConfusionMatrix -> [Float]
classwiseRecall mtx = [mtx A.! (i, i) / mtx `colSumAt` i | i <- [x1 .. x2]]
  where
    ((x1, y1), (x2, y2)) = A.bounds mtx


Convert one-hot encoded predictions back to class labels.
For example: [0.1, 0.8, 0.1] → 1 (because index 1 has highest value)



In [25]:
import Data.Function (on)
import Data.List (maximumBy)

reverseOneHot :: HT.Tensor -> [Int]
reverseOneHot tsr = map (fst . maximumBy (compare `on` snd) . zip [0 ..]) vals
  where
    vals = HT.asValue tsr :: [[Float]]


Evaluate on training set
========================



In [26]:
import qualified Data.Vector.Unboxed as VU 

putStrLn "....................................."
putStrLn "....................................."
putStrLn "Training Set Summary is as follows: "

let predTrain = reverseOneHot $ mlp trainedModel trainFeaturesTr
putStrLn "====== Confusion Matrix ========"
let confusionTrain = confusionMatrix 3 (VU.toList trainLabels) predTrain
putStrLn $ pprintMatrix confusionTrain

putStrLn "=========== Classwise Metrics ============="
print $ D.fromNamedColumns
    [ ("variety" , D.fromList (map (toEnum @Iris) [0 .. 2]))
    , ("precision", D.fromList (classwisePrecision confusionTrain))
    , ("recall", D.fromList (classwiseRecall confusionTrain))]


.....................................

.....................................

Training Set Summary is as follows:

====== Confusion Matrix ========

          0     1     2
    0 36.00  0.00  0.00
    1  0.00 30.00  1.00
    2  0.00  2.00 35.00

=========== Classwise Metrics =============

------------------------------------------
index |  variety   | precision |  recall  
------|------------|-----------|----------
 Int  |    Iris    |   Float   |   Float  
------|------------|-----------|----------
0     | Setosa     | 1.0       | 1.0      
1     | Versicolor | 0.9677419 | 0.9375   
2     | Virginica  | 0.9459459 | 0.9722222

Evaluate on test set
====================

This is the true test of our model - how well does it perform on data
it has never seen before?



In [27]:
putStrLn "....................................."
putStrLn "....................................."
putStrLn "Test Set Summary is as follows: "

let predTest = reverseOneHot $ mlp trainedModel testFeaturesTr
putStrLn "====== Confusion Matrix ========"
let confusionTest = confusionMatrix 3 (VU.toList testLabels) predTest
putStrLn $ pprintMatrix confusionTest

putStrLn "=========== Classwise Metrics ============="
print $ D.fromNamedColumns
    [ ("variety" , D.fromList (map (toEnum @Iris) [0 .. 2]))
    , ("precision", D.fromList (classwisePrecision confusionTest))
    , ("recall", D.fromList (classwiseRecall confusionTest))]


.....................................

.....................................

Test Set Summary is as follows:

====== Confusion Matrix ========

          0     1     2
    0 13.00  0.00  0.00
    1  1.00 16.00  1.00
    2  0.00  2.00 13.00

=========== Classwise Metrics =============

------------------------------------------
index |  variety   | precision |  recall  
------|------------|-----------|----------
 Int  |    Iris    |   Float   |   Float  
------|------------|-----------|----------
0     | Setosa     | 1.0       | 0.9285714
1     | Versicolor | 0.8888889 | 0.8888889
2     | Virginica  | 0.8666667 | 0.9285714

Conclusion
==========

This program demonstrates that Haskell can be used for machine learning
tasks just like Python! The functional style leads to concise, composable code.
